In [1]:
#ignore warnings
import warnings
warnings.filterwarnings('ignore')

In [2]:
import numpy as np
import pandas as pd
import json
import requests

In [3]:
shop = pd.read_csv('C:/Users/UserK/Downloads/소상공인시장진흥공단_상가(상권)정보_20230331/소상공인시장진흥공단_상가(상권)정보_서울_202303.csv',encoding='utf-8')
shop.shape

(474260, 39)

In [4]:
shop.columns

Index(['상가업소번호', '상호명', '지점명', '상권업종대분류코드', '상권업종대분류명', '상권업종중분류코드',
       '상권업종중분류명', '상권업종소분류코드', '상권업종소분류명', '표준산업분류코드', '표준산업분류명', '시도코드',
       '시도명', '시군구코드', '시군구명', '행정동코드', '행정동명', '법정동코드', '법정동명', '지번코드',
       '대지구분코드', '대지구분명', '지번본번지', '지번부번지', '지번주소', '도로명코드', '도로명', '건물본번지',
       '건물부번지', '건물관리번호', '건물명', '도로명주소', '구우편번호', '신우편번호', '동정보', '층정보',
       '호정보', '경도', '위도'],
      dtype='object')

In [5]:
shop_2023 = shop[['상호명', '지점명','상권업종소분류명','시도명','시군구명','행정동코드', '행정동명', '법정동코드', '법정동명','도로명', '위도','경도' ]]
shop_2023.head()

,상호명,지점명,상권업종소분류명,시도명,시군구명,행정동코드,행정동명,법정동코드,법정동명,도로명,위도,경도
0,영자's곱창,NaN,곱창 전골/구이,서울특별시,광진구,11215760,중곡3동,1121510100,중곡동,서울특별시 광진구 면목로,37.568503,127.082832
1,호텔클로이,NaN,여관/모텔,서울특별시,중구,11140590,광희동,1114014500,광희동1가,서울특별시 중구 을지로42길,37.565708,127.005849
2,나라차중계태권도장,NaN,태권도/무술학원,서울특별시,노원구,11350619,중계본동,1135010600,중계동,서울특별시 노원구 한글비석로,37.649226,127.076956
3,이상한스냅,NaN,사진촬영업,서울특별시,양천구,11470600,신월5동,1147010300,신월동,서울특별시 양천구 월정로,37.542117,126.828832
4,토끼정,NaN,백반/한정식,서울특별시,강서구,11500603,가양1동,1150010500,마곡동,서울특별시 강서구 마곡동로,37.560053,126.833022


In [6]:
shop_2023.isnull().sum()

상호명              0
지점명         473044
상권업종소분류명         0
시도명              0
시군구명             0
행정동코드            0
행정동명             0
법정동코드            0
법정동명             0
도로명              0
위도               0
경도               0
dtype: int64

In [7]:
shop_emd_count = shop_2023.groupby('행정동명').count()['상호명']
shop_emd_count

행정동명
가락1동     392
가락2동    1121
가락본동    1982
가리봉동     704
가산동     8830
        ... 
효창동      225
후암동      670
휘경1동     554
휘경2동     453
흑석동      948
Name: 상호명, Length: 425, dtype: int64

In [8]:
shop_2023.groupby('행정동명').agg('count')['상호명']

행정동명
가락1동     392
가락2동    1121
가락본동    1982
가리봉동     704
가산동     8830
        ... 
효창동      225
후암동      670
휘경1동     554
휘경2동     453
흑석동      948
Name: 상호명, Length: 425, dtype: int64

In [9]:
duk = shop_2023.groupby(['시군구명','행정동명'])['상호명'].agg('count').reset_index()
duk.to_clipboard()
duk.head()

,시군구명,행정동명,상호명
0,강남구,개포1동,91
1,강남구,개포2동,655
2,강남구,개포3동,304
3,강남구,개포4동,1357
4,강남구,논현1동,4585


In [10]:
duk['점포분류'] = duk['상호명'].apply(lambda x: '큰점포' if x > 1000 else '작은점포')
duk

,시군구명,행정동명,상호명,점포분류
0,강남구,개포1동,91,작은점포
1,강남구,개포2동,655,작은점포
2,강남구,개포3동,304,작은점포
3,강남구,개포4동,1357,큰점포
4,강남구,논현1동,4585,큰점포
...,...,...,...,...
421,중랑구,상봉2동,1338,큰점포
422,중랑구,신내1동,908,작은점포
423,중랑구,신내2동,469,작은점포
424,중랑구,중화1동,467,작은점포


In [11]:
shop_2023.pivot_table(index='행정동명',aggfunc='count',values=['상호명','위도'])

,상호명,위도
행정동명,,
가락1동,392,392
가락2동,1121,1121
가락본동,1982,1982
가리봉동,704,704
가산동,8830,8830
...,...,...
효창동,225,225
후암동,670,670
휘경1동,554,554


In [12]:
shop_2023.pivot_table(index='행정동명',aggfunc='count',values=['상호명','위도']).sort_values(by='위도', ascending=False).head(5)

,상호명,위도
행정동명,,
역삼1동,11944,11944
가산동,8830,8830
서교동,7656,7656
종로1.2.3.4가동,7290,7290
서초3동,6315,6315


In [121]:
df = pd.DataFrame({'foo': ['one', 'one', 'one', 'two', 'two',
                           'two'],
                   'bar': ['A', 'B', 'C', 'A', 'B', 'C'],
                   'baz': [1, 2, 3, 4, 5, 6],
                   'zoo': ['x', 'y', 'z', 'q', 'w', 't']})
df

,foo,bar,baz,zoo
0,one,A,1,x
1,one,B,2,y
2,one,C,3,z
3,two,A,4,q
4,two,B,5,w
5,two,C,6,t


In [14]:
df.pivot_table(['baz','bar'], index =['foo'],columns = 'bar',  aggfunc=['sum','count'])

sum       count                
    baz         baz       zoo      
bar   A  B  C     A  B  C   A  B  C
foo                                
one   1  2  3     1  1  1   1  1  1
two   4  5  6     1  1  1   1  1  1

In [15]:
df.pivot_table(['baz','bar'], index =['foo'],  aggfunc=['sum','count'])

sum count    
    baz   bar baz
foo              
one   6     3   3
two  15     3   3

In [16]:
df.pivot_table( index =['foo'],  aggfunc=['sum','count'])

sum count        
    baz   bar baz zoo
foo                  
one   6     3   3   3
two  15     3   3   3

In [27]:
df.pivot_table(index='foo', values='baz',aggfunc='sum')

,baz
foo,
one,6
two,15


In [18]:
df.set_index(['foo','bar'])['baz']

foo  bar
one  A      1
     B      2
     C      3
two  A      4
     B      5
     C      6
Name: baz, dtype: int64

In [19]:
df.set_index(['foo','bar'])['baz'].unstack()

bar,A,B,C
foo,,,
one,1,2,3
two,4,5,6


In [20]:
df.set_index(['foo','bar'])['baz'].unstack('foo')

foo,one,two
bar,,
A,1,4
B,2,5
C,3,6


In [122]:
df.set_index(['foo','bar'])['baz'].unstack('bar')

bar,A,B,C
foo,,,
one,1,2,3
two,4,5,6


In [22]:
shop_emd_count = shop_2023.pivot_table(index='법정동명',aggfunc='count',values=['상호명']
shop_emd_count.head()

,상호명
행정동명,
가락1동,392
가락2동,1121
가락본동,1982
가리봉동,704
가산동,8830


In [37]:
shop_emd_count[:3].reset_index()

,행정동명,상호명
0,가락1동,392
1,가락2동,1121
2,가락본동,1982


In [ ]:
#my_list = ['a','b','c','d']
my_list = 'abcd'
for index, value in enumerate(my_list):
    print('{0},{1}'.format(index, value))

In [38]:
df

,foo,bar,baz,zoo
0,one,A,1,x
1,one,B,2,y
2,one,C,3,z
3,two,A,4,q
4,two,B,5,w
5,two,C,6,t


In [42]:
df.loc[:3,['bar','zoo']]

,bar,zoo
0,A,x
1,B,y
2,C,z
3,A,q


In [46]:
df.sort_values(['foo','bar'],ascending=False)

,foo,bar,baz,zoo
5,two,C,6,t
4,two,B,5,w
3,two,A,4,q
2,one,C,3,z
1,one,B,2,y
0,one,A,1,x


In [48]:
df.sum()

foo    oneoneonetwotwotwo
bar                ABCABC
baz                    21
zoo                xyzqwt
dtype: object

In [50]:
df.values

array([['one', 'A', 1, 'x'],
       ['one', 'B', 2, 'y'],
       ['one', 'C', 3, 'z'],
       ['two', 'A', 4, 'q'],
       ['two', 'B', 5, 'w'],
       ['two', 'C', 6, 't']], dtype=object)

In [53]:
# 판다는 기본 numpy
df['baz']> 3

0    False
1    False
2    False
3     True
4     True
5     True
Name: baz, dtype: bool

In [60]:
df.iloc[:3,:2]

,foo,bar
0,one,A
1,one,B
2,one,C


In [62]:
df.drop('bar',axis=1)

,foo,baz,zoo
0,one,1,x
1,one,2,y
2,one,3,z
3,two,4,q
4,two,5,w
5,two,6,t


In [65]:
df.set_index(['foo','bar'], drop=False) # 컬럼 남고 인덱스 설정

foo bar  baz zoo
foo bar                  
one A    one   A    1   x
    B    one   B    2   y
    C    one   C    3   z
two A    two   A    4   q
    B    two   B    5   w
    C    two   C    6   t

In [123]:
df1 = df.set_index(['foo','bar'], append=True)  # 기존 인덱스에 추가로 설정
df1

,,,baz,zoo
,foo,bar,,
0,one,A,1,x
1,one,B,2,y
2,one,C,3,z
3,two,A,4,q
4,two,B,5,w
5,two,C,6,t


In [126]:
df1.reset_index()

,level_0,foo,bar,baz,zoo
0,0,one,A,1,x
1,1,one,B,2,y
2,2,one,C,3,z
3,3,two,A,4,q
4,4,two,B,5,w
5,5,two,C,6,t


In [69]:
df.T

,0,1,2,3,4,5
foo,one,one,one,two,two,two
bar,A,B,C,A,B,C
baz,1,2,3,4,5,6
zoo,x,y,z,q,w,t


In [70]:
df.T.reset_index()

,index,0,1,2,3,4,5
0,foo,one,one,one,two,two,two
1,bar,A,B,C,A,B,C
2,baz,1,2,3,4,5,6
3,zoo,x,y,z,q,w,t


In [71]:
df.T.reset_index().T

,0,1,2,3
index,foo,bar,baz,zoo
0,one,A,1,x
1,one,B,2,y
2,one,C,3,z
3,two,A,4,q
4,two,B,5,w
5,two,C,6,t


In [76]:
df.reindex(columns=['baz','foo','duk'],fill_value=3)

,baz,foo,duk
0,1,one,3
1,2,one,3
2,3,one,3
3,4,two,3
4,5,two,3
5,6,two,3


In [77]:
df.sort_index(ascending = False)

,foo,bar,baz,zoo
5,two,C,6,t
4,two,B,5,w
3,two,A,4,q
2,one,C,3,z
1,one,B,2,y
0,one,A,1,x


In [78]:
df

,foo,bar,baz,zoo
0,one,A,1,x
1,one,B,2,y
2,one,C,3,z
3,two,A,4,q
4,two,B,5,w
5,two,C,6,t


In [111]:
# stack -> reset_index  ==> unpivot 이다.  즉 스텍으로 전체 인덱스 화 하여 series 화 후에 인덱스 칼럼 만들어 주는 것과 같음

In [161]:
val = [[83, 93, 99], [77, 88, 91], [92, 78, 95]]
idx = ["권보아", "김범수", "박효신"]
df = pd.DataFrame(val, index=idx, columns=["국어", "수학", "과학"])
val1 = [[20, 10], [5, 25], [22, 8], [12, 18]]
idx1 = [["1반", "1반", "2반", "2반"], ["남", "여","남","여"]]
df1 = pd.DataFrame(val1,index=idx1, columns=["도시락", "샌드위치"])
df2 = df1.copy().unstack()
df3 = df1.copy()
df3["업체"] = ["CJ푸드", "신세계", "신세계", "CJ푸드"]
df4 = df3.reset_index().rename(columns={"level_0":"반", "level_1":"성별"}).copy()

In [128]:
df

,국어,수학,과학
권보아,83,93,99
김범수,77,88,91
박효신,92,78,95


In [129]:
df.stack()

권보아  국어    83
     수학    93
     과학    99
김범수  국어    77
     수학    88
     과학    91
박효신  국어    92
     수학    78
     과학    95
dtype: int64

In [130]:
df.stack().reset_index()

,level_0,level_1,0
0,권보아,국어,83
1,권보아,수학,93
2,권보아,과학,99
3,김범수,국어,77
4,김범수,수학,88
5,김범수,과학,91
6,박효신,국어,92
7,박효신,수학,78
8,박효신,과학,95


In [134]:
df = df.stack().reset_index()
df.columns = ['이름', '과목', '점수']
df

,이름,과목,점수
0,권보아,국어,83
1,권보아,수학,93
2,권보아,과학,99
3,김범수,국어,77
4,김범수,수학,88
5,김범수,과학,91
6,박효신,국어,92
7,박효신,수학,78
8,박효신,과학,95


In [135]:
df1

도시락  샌드위치
1반 남   20    10
   여    5    25
2반 남   22     8
   여   12    18

In [142]:
df1 = df1.stack().reset_index()
df1.columns = ['반','성별','제품','수량']
df1

,반,성별,제품,수량
0,1반,남,도시락,20
1,1반,남,샌드위치,10
2,1반,여,도시락,5
3,1반,여,샌드위치,25
4,2반,남,도시락,22
5,2반,남,샌드위치,8
6,2반,여,도시락,12
7,2반,여,샌드위치,18


In [144]:
df2

도시락     샌드위치    
     남   여    남   여
1반  20   5   10  25
2반  22  12    8  18

In [158]:
df22 = df2
df22

도시락     샌드위치    
     남   여    남   여
1반  20   5   10  25
2반  22  12    8  18

In [147]:
df2 = df2.stack().stack().reset_index()
df2.columns = ['반','성별','제품','수량']
df2

,반,성별,제품,수량
0,1반,남,도시락,20
1,1반,남,샌드위치,10
2,1반,여,도시락,5
3,1반,여,샌드위치,25
4,2반,남,도시락,22
5,2반,남,샌드위치,8
6,2반,여,도시락,12
7,2반,여,샌드위치,18


In [159]:
df22 = df22.stack(level=[0,1]).reset_index()
df22.columns = ['반','성별','제품','수량']
df22

,반,성별,제품,수량
0,1반,도시락,남,20
1,1반,도시락,여,5
2,1반,샌드위치,남,10
3,1반,샌드위치,여,25
4,2반,도시락,남,22
5,2반,도시락,여,12
6,2반,샌드위치,남,8
7,2반,샌드위치,여,18


In [162]:
df3

도시락  샌드위치    업체
1반 남   20    10  CJ푸드
   여    5    25   신세계
2반 남   22     8   신세계
   여   12    18  CJ푸드

In [164]:
df3.set_index('업체',append = True)

도시락  샌드위치
     업체             
1반 남 CJ푸드   20    10
   여 신세계     5    25
2반 남 신세계    22     8
   여 CJ푸드   12    18

In [165]:
df3.set_index('업체',append = True).stack().reset_index()

,level_0,level_1,업체,level_3,0
0,1반,남,CJ푸드,도시락,20
1,1반,남,CJ푸드,샌드위치,10
2,1반,여,신세계,도시락,5
3,1반,여,신세계,샌드위치,25
4,2반,남,신세계,도시락,22
5,2반,남,신세계,샌드위치,8
6,2반,여,CJ푸드,도시락,12
7,2반,여,CJ푸드,샌드위치,18


In [166]:
df3 = df3.set_index('업체',append = True).stack().reset_index()
df3.columns = ['반','성별','업체', '제품','수량']
df3

,반,성별,업체,제품,수량
0,1반,남,CJ푸드,도시락,20
1,1반,남,CJ푸드,샌드위치,10
2,1반,여,신세계,도시락,5
3,1반,여,신세계,샌드위치,25
4,2반,남,신세계,도시락,22
5,2반,남,신세계,샌드위치,8
6,2반,여,CJ푸드,도시락,12
7,2반,여,CJ푸드,샌드위치,18


In [174]:
df3.value_counts(['업체','제품'])

업체    제품  
CJ푸드  도시락     2
      샌드위치    2
신세계   도시락     2
      샌드위치    2
dtype: int64

In [182]:
df3.value_counts('제품').iloc[:2].index

Index(['도시락', '샌드위치'], dtype='object', name='제품')

In [185]:
df3.loc[df3['제품'].isin(df3.value_counts('제품').iloc[:1].index)]  #  상위 3 제품 만 선택적 처리

,반,성별,업체,제품,수량
0,1반,남,CJ푸드,도시락,20
2,1반,여,신세계,도시락,5
4,2반,남,신세계,도시락,22
6,2반,여,CJ푸드,도시락,12
